In [ ]:
# 1 - New Assets Handler

from pgmask.dataframelayer import DataframeLayer
from pgmask.basiclayer import BasicLayer

#Production
#POSTGRES_USER = os.environ['POSTGRES_USER']
#POSTGRES_PASSWORD = os.environ['POSTGRES_PASSWORD']
#DB_HOST = os.environ['DB_HOST']
#PG_PORT = os.environ['PG_PORT']
#POSTGRES_DB = os.environ['POSTGRES_DB']

#Dev-new    
POSTGRES_USER = 'pandazord'
POSTGRES_PASSWORD = 'QBBV9E%pcYKHUcjj'
DB_HOST = 'localhost'
PG_PORT = 5432
POSTGRES_DB = 'pandazord_database'

class AssetsHandler:

    def __init__(self, exchange_name, asset_symbol):

        self._PGDF = DataframeLayer(POSTGRES_USER, POSTGRES_PASSWORD, DB_HOST, PG_PORT, POSTGRES_DB)
        self._PGBL = BasicLayer(POSTGRES_USER, POSTGRES_PASSWORD, DB_HOST, PG_PORT, POSTGRES_DB)
        self._exchange_name = exchange_name
        self._asset_symbol = asset_symbol
    
    def get_dataframe(self):
        
        self.asset_df = self._PGDF.read_entries_list(table_name=(self._exchange_name + '_assets'),
                                                     column_name= 'asset_symbol',
                                                     entries_list=self._asset_symbol.split(sep=','))

        return self.asset_df
    
    def operational_parameters(self):
        
        self.op_param = self.get_dataframe().iloc[0].operational_parameters
        return self.op_param
    

class Position:

    def __init__(self, current_position):
        self._side = current_position['side']
        self._size = current_position['size']
        self._target_price = current_position['target_price']

    @property
    def side(self):
        return self._side

    @property
    def size(self):
        return self._size

    @property
    def target_price(self):
        return self._target_price

    @side.setter
    def side(self, side_to_set):
        side_list = ['closed', 'long', 'short']
        error_message = 'Incorrect! Side must be '
        error_append = ''

        if (side_to_set not in side_list):
            for item in side_list[:-1]:
                error_append+= "'" + item + "'" + ', '

            error_message = error_message + error_append + 'or ' + "'" + side_list[-1] + "'" + '.'

            print(error_message) #TODO: Tratar exceção sem print
        else:
            self._side = side_to_set

    @size.setter
    def size(self, size_to_set):
        numerical = (isinstance(size_to_set, float) or isinstance(size_to_set, int))

        if (numerical):
            if (size_to_set < 0.0):
                print ('Invalid. Size must not be negative.') #TODO: Tratar exceção sem print

            else:
                self._size = size_to_set

        else:
            print("Invalid. Must be a numerical ('float' or 'int') type." ) #TODO: Tratar exceção sem print

    @target_price.setter
    def target_price(self, target_price_to_set):
        numerical = (isinstance(target_price_to_set, float) or isinstance(target_price_to_set, int)) 

        if (numerical):
            if (target_price_to_set < 0.0):
                print ('Invalid. Target price must not be negative.') #TODO: Tratar exceção sem print

            else:
                self._target_price = target_price_to_set

        else:
            print("Invalid. Must be a numerical ('float' or 'int') type." ) #TODO: Tratar exceção sem print

class BackingTestAssetPosition(Position):
    pass

class RealTradePosition(Position):
    pass

    #TODO: Atualizar este método
    #Por se tratar de um objeto json, possivelmente terá dde ser todo carregado e atualizado 
    #apenas o valor da chave desejada. Como fazer isto de modo "DRY", para qualquer chave dada?
    def update_position(self, field, update_to):

        update_status = self.PGBL.update_entry(table_name = self.exchange_name + '_assets', 
        pk_field = 'asset_symbol', 
        pk_value = self.asset_symbol, 
        field_to_update = field, 
        new_field_value = update_to)

        return update_status

In [ ]:
#Trader v0.2.0
#from modules.assets_handler import AssetsHandler
from pydoc import locate

class Trader:
    ## Common instantiation, trader and backing test
    def __init__(self, exchange_name, asset_symbol):
        
        self.Asset = AssetsHandler(exchange_name, asset_symbol)
        self.strategy_param = self.Asset.operational_parameters()['strategy']['parameters']
        self.stop_param = self.Asset.operational_parameters()['stop_loss']['parameters']
        strategy_name = self.Asset.operational_parameters()['strategy']['name']
        stop_name = self.Asset.operational_parameters()['stop_loss']['name']
        self.Strategy = locate('modules.strategies.' + strategy_name)(strategy_param)
        self.StopLoss = locate('modules.stop_loss.' + stop_name)(stop_param)
        self.OrderHandler = locate('modules.order_handler' + exchange_name.capitalize() + 'Handler')
        self.KlinesHandler = locate('modules.klines_handler.' + exchange_name.capitalize() + 'FromDb')()
        
    def for_real(self):
        pass
    
    def backing_test(self):
        pass
        

In [ ]:
#Testing Klines_handler
from pydoc import locate

exchange_name, asset_symbol = 'binance', 'BTCUSDT'

KlinesHandler = locate('modules.klines_handler.' + exchange_name.capitalize() + 'FromDb')()

klines_to_strategy_analisys = KlinesHandler.get_latest(asset_symbol=asset_symbol, 
                                                       candle_interval='1h', 
                                                       number_of_candles=100)

In [ ]:
klines_to_strategy_analisys

In [ ]:
asset_symbol

In [ ]:
KlinesHandler().get_latest

In [ ]:
KlinesHandler = locate('modules.klines_handler.' + exchange_name.capitalize() + 'FromDb')

In [ ]:
KlinesHandler.all_latest_one_minute('BTCUSDT')

In [ ]:
exchange_name, asset_symbol = 'binance', 'BTCUSDT'

Trader(exchange_name, asset_symbol)

In [ ]:
exchange_name, asset_symbol = 'binance', 'BTCUSDT'

In [ ]:
Asset = AssetsHandler(exchange_name, asset_symbol)

In [ ]:
Asset.operational_parameters()

In [ ]:
real_current_position = Asset.get_dataframe().iloc[0].position

In [ ]:
bt_current_position = {'side': 'short', 'size': 0.0, 'target_price': 0.0}

In [ ]:
Position = RealTradePosition(bt_current_position)

In [ ]:
Position.side = 'closed'

In [ ]:
Position.side

In [ ]:
Asset.operational_parameters()